In [1]:
import pandas as pd
import numpy as np

###import pybrain as pb
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.datasets import SupervisedDataSet
from pybrain.structure import SigmoidLayer
from pybrain.structure import LinearLayer
from pybrain.structure import SoftmaxLayer

from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

import pickle
import json

import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz

pd.set_option('display.precision', 3);
%matplotlib inline

import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import json

# Загрузка данных

In [2]:
dataTEST = pd.read_csv("./WV_test.csv", # путь к файлу, используй автодотолнение
                   sep=',', # разделитель данных в файле
                   header=0 # номер строки с заголовками. header='None', если заголовки отсутствуют
                  )
dataTEST_Norm = pd.read_csv("./WVNorm_test.csv", # путь к файлу, используй автодотолнение
                   sep=',', # разделитель данных в файле
                   header=0 # номер строки с заголовками. header='None', если заголовки отсутствуют
                  )

In [3]:
dataTEST.shape

(2871, 5)

In [4]:
dataTEST.columns

Index([u'Temperature', u'Vacuum', u'Pressure', u'Humidity', u'EnergyOutput'], dtype='object')

In [5]:
features = [u'Temperature', u'Vacuum']
target = [u'EnergyOutput']

In [6]:
dataTEST[:2]

,Temperature,Vacuum,Pressure,Humidity,EnergyOutput
0,12.64,41.49,1019.78,94.00,468.78
1,27.08,62.91,1011.90,62.58,442.91


In [7]:
dataTEST_Norm[:2]

,Temperature,Vacuum,Pressure,Humidity,EnergyOutput
0,0.307,0.287,0.665,0.917,0.643
1,0.716,0.668,0.470,0.496,0.300


In [8]:
dataTEST.describe()

,Temperature,Vacuum,Pressure,Humidity,EnergyOutput
count,2871.000,2871.000,2871.000,2871.000,2871.000
mean,19.538,54.029,1013.136,73.350,454.627
std,7.453,12.615,5.879,14.734,17.053
min,2.580,25.360,993.820,30.590,421.570
25%,13.430,41.770,1008.850,63.390,439.890
50%,20.080,51.190,1012.830,75.040,452.060
75%,25.580,66.480,1017.215,85.060,468.615
max,35.770,80.180,1033.190,100.150,494.840


In [9]:
x_test = dataTEST.drop(columns=target)
y_test = dataTEST[target]
x_test_Norm = dataTEST_Norm.drop(columns=target)
y_test_Norm = dataTEST_Norm[target]

In [10]:
x_test[:2]

,Temperature,Vacuum,Pressure,Humidity
0,12.64,41.49,1019.78,94.00
1,27.08,62.91,1011.90,62.58


In [11]:
x_test_Norm[:2]

,Temperature,Vacuum,Pressure,Humidity
0,0.307,0.287,0.665,0.917
1,0.716,0.668,0.470,0.496


# Шкалы стандартизации. Коэфф.

In [12]:
'''
Temperature =     8.340000 истинное ~  [[0.18498584]] шкалированное 
Temperature     =     0.184986 шкалированное  ~  [[8.34000015]] истинное 

Temperature
data_min_    =  [1.81]
data_max_    =  [37.11]
min_ =  [-0.05127479]
scale_ =  [0.02832861]
data_range_  =  [35.3]
'''
# Temperature
newScaler_Temperature = MinMaxScaler(feature_range=(0, 1))

newScaler_Temperature.data_min_    =  [1.81]
newScaler_Temperature.data_max_    =  [37.11]
newScaler_Temperature.min_ =  [-0.05127479]
newScaler_Temperature.scale_ =  [0.02832861]
newScaler_Temperature.data_range_  =  [35.3]

# .transform( [[8.340000]] ) - привести значение 8.340000 к заданному диапазону
print "Нормализация ",newScaler_Temperature.transform( [[8.340000]] )

# .inverse_transform( [[0.18498584]] ) - привести значение 0.18498584 к исходному диапазону
print "Денормализация ",newScaler_Temperature.inverse_transform( [[0.18498584]])

Нормализация  [[0.18498582]]
Денормализация  [[8.3400008]]


In [13]:
'''
Vacuum
data_min_    =  [25.36]
data_max_    =  [81.56]
min_ =  [-0.45124555]
scale_ =  [0.01779359]
data_range_  =  [56.2]

Vacuum =    40.770000 истинное ~  [[0.27419929]] шкалированное 
Vacuum     =     0.274199 шкалированное  ~  [[40.7700001]] истинное 

'''
# Temperature
newScaler_Vacuum = MinMaxScaler(feature_range=(0, 1))

newScaler_Vacuum.data_min_    =  [25.36]
newScaler_Vacuum.data_max_    =  [81.56]
newScaler_Vacuum.min_ =  [-0.45124555]
newScaler_Vacuum.scale_ =  [0.01779359]
newScaler_Vacuum.data_range_  =  [56.2]

# .transform( [[40.770000]] ) - привести значение 40.770000 к заданному диапазону
print "Нормализация ",newScaler_Vacuum.transform( [[40.770000]] )

# .inverse_transform( [[0.274199]] ) - привести значение 0.274199 к исходному диапазону
print "Денормализация ",newScaler_Vacuum.inverse_transform( [[0.274199]])

Нормализация  [[0.27419911]]
Денормализация  [[40.76999358]]


In [14]:
'''
EnergyOutput
data_min_    =  [420.26]
data_max_    =  [495.76]
min_ =  [-5.56635762]
scale_ =  [0.01324503]
data_range_  =  [75.5]

EnergyOutput =    480.48 истинное ~  [[0.79761439]] шкалированное 
EnergyOutput     =     0.79761439 шкалированное  ~  [[480.47999967]] истинное 

'''
# EnergyOutput
newScaler_EnergyOutput = MinMaxScaler(feature_range=(0, 1))

newScaler_EnergyOutput.data_min_    =  [420.26]
newScaler_EnergyOutput.data_max_    = [495.76]
newScaler_EnergyOutput.min_ =  [-5.56635762]
newScaler_EnergyOutput.scale_ =  [0.01324503]
newScaler_EnergyOutput.data_range_  =  [75.5]

# .transform( [[480.48]] ) - привести значение480.48 к заданному диапазону
print "Нормализация ",newScaler_EnergyOutput.transform( [[480.48]] )

# .inverse_transform( [[0.79761439]] ) - привести значение 0.79761439 к исходному диапазону
print "Денормализация ",newScaler_EnergyOutput.inverse_transform( [[0.79761439]])

Нормализация  [[0.79761439]]
Денормализация  [[480.47999967]]


In [15]:
descriptionJson = open("./model_description.json",'r')
description = json.load(descriptionJson)

# Нейронная сеть. Файл pickle.

## Модель 2 вх., 1 вых.

In [16]:
x_sample_2 = x_test_Norm[features][0:1]
x_sample_2.values[0]

array([0.30679887, 0.28701068])

In [17]:
import pickle
fileName_net2 = "model_Net_3to1.data_pickle"

with open(fileName_net2, 'rb') as f:
    net2 = pickle.load(f)
print "Результат проfверки. Нейронная сеть 2 вх., 1 вых. Файл pickle."
print net2.activate(x_sample_2.values[0])

Результат проfверки. Нейронная сеть 2 вх., 1 вых. Файл pickle.
[0.67374703]


In [18]:
resultNet2 = net2.activate( x_sample_2.values[0] )
discNet2 = description['neuroNet2'] 

## Модель 12 вх., 1 вых.

In [19]:
x_sample_12 = x_test_Norm[0:1]
x_sample_12.values[0]

array([0.30679887, 0.28701068, 0.66542935, 0.91742627])

In [20]:
import pickle
# имя файла
fileName_net12 = "model_Net_WV.data_pickle"

with open(fileName_net12, 'rb') as f:
    net12 = pickle.load(f)
print "Результат проверки. Нейронная сеть 2 вх., 1 вых. Файл pickle."
print net12.activate( x_sample_12.values[0] )

Результат проверки. Нейронная сеть 2 вх., 1 вых. Файл pickle.
[0.62228886]


In [21]:
resultNet12 = net12.activate( x_sample_12.values[0] )
discNet12 = description['neuroNet12']

## Линейная регрессия. Коэфф.

In [22]:
model_LinRegrMult           = linear_model.LinearRegression()
model_LinRegrMult.coef_      = np.array([-0.793, -0.244])
model_LinRegrMult.intercept_ = np.array([0.978])
y_pred_LinRegrMult_CLEAR           = model_LinRegrMult.predict(  x_test_Norm[features]  )


r2_Lin = metrics.r2_score(y_test_Norm, y_pred_LinRegrMult_CLEAR)
mse_Lin = metrics.mean_squared_error(y_test_Norm, y_pred_LinRegrMult_CLEAR)

print "R2   = ", r2_Lin
print "MAE  = ", mse_Lin, newScaler_EnergyOutput.inverse_transform( [[ r2_Lin ]])
print "RMSE  = {} {:,.3f} ".format( mse_Lin,newScaler_EnergyOutput.inverse_transform( [[ r2_Lin ]])[0][0])

R2   =  0.917876348482411
MAE  =  0.004187942811827928 [[489.55977967]]
RMSE  = 0.00418794281183 489.560 


In [24]:
print "Результат проверки. Линейная регрессия. Коэфф."
print "Проверочное ", y_test_Norm.iloc[0:1].values
print "Вычисленное ", y_pred_LinRegrMult_CLEAR[1] 

print "Проверочное ", newScaler_EnergyOutput.inverse_transform(y_test_Norm.iloc[0:1].values)
print "Вычисленное ", newScaler_EnergyOutput.inverse_transform([[y_pred_LinRegrMult_CLEAR[1]]])

Результат проверки. Линейная регрессия. Коэфф.
Проверочное  [[0.64264901]]
Вычисленное  0.24729136027743892
Проверочное  [[468.78011047]]
Вычисленное  [[438.93060116]]


## Линейная регрессия. Файл pickle.

In [26]:
import pickle
fileName_LinRegrMult = "model_LinRegrMult.data_pickle"

with open(fileName_LinRegrMult, 'rb') as f:
    model_LinRegrMult = pickle.load(f)

y_pred_LinRegrMult_CLEAR = model_LinRegrMult.predict(  x_test_Norm[features]  )
discLinear = description['linear']
print "Результат проверки. Линейная регрессия. Файл pickle."

print "y_pred Исходной, ", "y_pred Загруженной, ", "delta, ", "delta < 0.000001"

for i in range (0, 10):
    delta = (y_test_Norm.iloc[i:i+1].values  - y_pred_LinRegrMult_CLEAR[i] ) 
    print y_test_Norm.iloc[i:i+1].values , y_pred_LinRegrMult_CLEAR[i], delta, delta < 0.000001

Результат проверки. Линейная регрессия. Файл pickle.
y_pred Исходной,  y_pred Загруженной,  delta,  delta < 0.000001
[[0.64264901]] [0.66501902] [[-0.02237001]] [[ True]]
[[0.3]] [0.24758593] [[0.05241407]] [[False]]
[[0.51933775]] [0.57798572] [[-0.05864797]] [[ True]]
[[0.29933775]] [0.27548388] [[0.02385387]] [[False]]
[[0.42238411]] [0.36842951] [[0.05395459]] [[False]]
[[0.54569536]] [0.59208788] [[-0.04639252]] [[ True]]
[[0.32887417]] [0.3807259] [[-0.05185173]] [[ True]]
[[0.38966887]] [0.40485561] [[-0.01518673]] [[ True]]
[[0.86993377]] [0.82008826] [[0.04984551]] [[False]]
[[0.48476821]] [0.50108903] [[-0.01632082]] [[ True]]


## Дерево регрессии. Файл pickle.

In [28]:
import pickle
fileName_treeRegr_d6 = "model_treeRegr_d6.data_pickle"

with open(fileName_LinRegrMult, 'rb') as f:
    treeRegr_d6 = pickle.load(f)

y_pred_treeRegr_d6 = treeRegr_d6.predict(  x_test_Norm[features]  )
discTree = description['tree']
print "Результат проверки. Дерево регрессии"

print "y_pred Исходной, ", "y_pred Загруженной, ", "delta, ", "delta < 0.000001"

for i in range (0, 10):
    delta = (y_test_Norm.iloc[i:i+1].values  - y_pred_treeRegr_d6[i] ) 
    print y_test_Norm.iloc[i:i+1].values , y_pred_treeRegr_d6[i], delta, delta < 0.000001

Результат проверки. Дерево регрессии
y_pred Исходной,  y_pred Загруженной,  delta,  delta < 0.000001
[[0.64264901]] [0.66501902] [[-0.02237001]] [[ True]]
[[0.3]] [0.24758593] [[0.05241407]] [[False]]
[[0.51933775]] [0.57798572] [[-0.05864797]] [[ True]]
[[0.29933775]] [0.27548388] [[0.02385387]] [[False]]
[[0.42238411]] [0.36842951] [[0.05395459]] [[False]]
[[0.54569536]] [0.59208788] [[-0.04639252]] [[ True]]
[[0.32887417]] [0.3807259] [[-0.05185173]] [[ True]]
[[0.38966887]] [0.40485561] [[-0.01518673]] [[ True]]
[[0.86993377]] [0.82008826] [[0.04984551]] [[False]]
[[0.48476821]] [0.50108903] [[-0.01632082]] [[ True]]


In [30]:
print y_pred_treeRegr_d6[0:1], newScaler_EnergyOutput.inverse_transform(y_pred_treeRegr_d6[0:1])

 [[0.66501902]] [[470.46904677]]


In [33]:
print "Результат проверки. Нейронная сеть 2 вх., 1 вых. Файл pickle."
print resultNet2 ," | ", newScaler_EnergyOutput.inverse_transform([resultNet2])
print "\n"
for key in discNet2:
  print key," ",discNet2[key]

print "\n"
print "Результат проверки. Нейронная сеть 12 вх., 1 вых. Файл pickle."
print resultNet12," | ",newScaler_EnergyOutput.inverse_transform([resultNet12]),"\n"
for key in discNet12:
  print "@", key," ",discNet12[key]

print "\n"

print "Результат проверки. Линейная регрессия. Коэфф."
print y_test_Norm.iloc[0:1].values," | ",newScaler_EnergyOutput.inverse_transform(y_test_Norm.iloc[0:1].values)
#print "Вычисленное ", y_pred_LinRegrMult_CLEAR[1]," | "#, newScaler_quality.inverse_transform([[y_pred_LinRegrMult_CLEAR[1]]])
print "\n"
print "Результат проверки. Линейная регрессия. Файл pickle."
print y_pred_LinRegrMult_CLEAR[0:1], "|", newScaler_EnergyOutput.inverse_transform(y_pred_LinRegrMult_CLEAR[0:1])

print "\n"
for key in discLinear:
  print key," ",discLinear[key]
print "\n"
print "Результат проверки. Дерево регрессии"
print y_pred_treeRegr_d6[0:1], "|", newScaler_EnergyOutput.inverse_transform(y_pred_treeRegr_d6[0:1])

print "\n"

for key in discTree:
  print key," ",discTree[key]

Результат проверки. Нейронная сеть 2 вх., 1 вых. Файл pickle.
[0.67374703]  |  [[471.12801185]]


dataNormalised   True
target   [u'EnergyOutput']
r2   0.92741328581
name   Нейросеть 2вх, 1 вых. 75 итераций
mae   0.0477412097737
features   [u'Temperature', u'Vacuum']
mse   0.0037016012112
typeML   Нейронная сеть


Результат проверки. Нейронная сеть 12 вх., 1 вых. Файл pickle.
[0.62228886]  |  [[467.24291904]] 



Результат проверки. Линейная регрессия. Коэфф.
[[0.64264901]]  |  [[468.78011047]]


Результат проверки. Линейная регрессия. Файл pickle.
[[0.66501902]] | [[470.46904677]]


dataNormalised   True
target   [u'temp']
r2   0.917872778718
name   Множественная
mae   0.051046196666
features   [u'RH', u'ISI']
mse   0.00418812485402
typeML   Линейная регрессия


Результат проверки. Дерево регрессии
[[0.66501902]] | [[470.46904677]]


dataNormalised   True
target   [u'temp']
r2   0.933011173534
name   Дерево регрессии
mae   0.0441535536356
features   [u'RH', u'ISI']
mse   0.00341613370

# Выводы

Был разработан проект сегмента базы знаний интеллектуальной системы, содержащей набор моделей, сформированных с помощью различных методов машинного обучения (модели линейной множественной регрессии, деревья решений, нейронные сети). Сопроводительная информация о моделях была сохранена в предыдущей работе в формате json.

Полученные результаты:

Нейронная сеть 2 вх., 1 вых. Файл pickle. [0.67374703]  |  [471.12801185]

Нейронная сеть 12 вх., 1 вых. Файл pickle. [0.62228886]  |  [467.24291904]

Линейная регрессия. Коэфф. [0.64264901]  |  [468.78011047]]

Линейная регрессия. Файл pickle.[0.66501902]] | [470.46904677]

Дерево регрессии [0.66501902] | [470.46904677]